1 - Import Libraries

In [1]:
import yfinance
import pandas as pd
import warnings # for ignoring warnings
warnings.filterwarnings("ignore")

2 - Download Data

In [2]:
raw_data = yfinance.download (tickers = "^BSESN ^GSPC ^FTSE ^N225 ^GDAXI", #The time series we are interested in - (in our case, these are the S&P, FTSE, NIKKEI and DAX)
                                start = "2000-01-01", #The starting date of our data set
                                end = "2022-06-01", #The ending date of our data set (at the time of upload, this is the current date)
                                interval = "1d", #The distance in time between two recorded observations. Since we're using daily closing prices, we set it equal to "1d", which indicates 1 day. 
                                group_by = 'ticker', #The way we want to group the scraped data. Usually we want it to be "ticker", so that we have all the information about a time series in 1 variable.
                                auto_adjust = True, #Automatically adjuss the closing prices for each period. 
                                treads = True) #Whether to use threads for mass downloading. 

[*********************100%***********************]  5 of 5 completed


In [3]:
raw_data.head()

^N225                                                   \
                    Open          High           Low         Close Volume   
Date                                                                        
1999-12-31           NaN           NaN           NaN           NaN    NaN   
2000-01-03           NaN           NaN           NaN           NaN    NaN   
2000-01-04  18937.449219  19187.609375  18937.449219  19002.859375    0.0   
2000-01-05  19003.509766  19003.509766  18221.820312  18542.550781    0.0   
2000-01-06  18574.009766  18582.740234  18168.269531  18168.269531    0.0   

                 ^GDAXI                                                     \
                   Open         High          Low        Close      Volume   
Date                                                                         
1999-12-31          NaN          NaN          NaN          NaN         NaN   
2000-01-03  6961.720215  7159.330078  6720.870117  6750.759766  43072500.0   
2000-01-04  6747.240234  6755.359863  6510.459961  6586.950195  46678400.0   
2000-01-05  6585.850098  6585.850098  6388.910156  6502.069824  52682800.0   
2000-01-06  6501.450195  6539.310059  6402.629883  6474.919922  41180600.0   

            ...       ^BSESN                                                \
            ...         Open         High          Low        Close Volume   
Date        ...                                                              
1999-12-31  ...          NaN          NaN          NaN          NaN    NaN   
2000-01-03  ...  5209.540039  5384.660156  5209.540039  5375.109863    0.0   
2000-01-04  ...  5533.979980  5533.979980  5376.430176  5491.009766    0.0   
2000-01-05  ...  5265.089844  5464.350098  5184.479980  5357.000000    0.0   
2000-01-06  ...  5424.209961  5489.859863  5391.330078  5421.529785    0.0   

                  ^FTSE                                                      
                   Open         High          Low        Close       Volume  
Date                                                                         
1999-12-31          NaN          NaN          NaN          NaN          NaN  
2000-01-03          NaN          NaN          NaN          NaN          NaN  
2000-01-04  6930.200195  6930.200195  6662.899902  6665.899902  633449000.0  
2000-01-05  6665.899902  6665.899902  6500.399902  6535.899902  670234000.0  
2000-01-06  6535.899902  6547.299805  6424.399902  6447.200195  785532000.0  

[5 rows x 25 columns]

3 - General Preprocessing

In [4]:
df = pd.DataFrame([])
df['bse'] = raw_data['^BSESN'].Close
df['spx'] = raw_data['^GSPC'].Close
df['dax'] = raw_data['^GDAXI'].Close
df['ftse'] = raw_data['^FTSE'].Close
df['nikkei'] = raw_data['^N225'].Close
df.head()

,bse,spx,dax,ftse,nikkei
Date,,,,,
1999-12-31,NaN,1469.250000,NaN,NaN,NaN
2000-01-03,5375.109863,1455.219971,6750.759766,NaN,NaN
2000-01-04,5491.009766,1399.420044,6586.950195,6665.899902,19002.859375
2000-01-05,5357.000000,1402.109985,6502.069824,6535.899902,18542.550781
2000-01-06,5421.529785,1403.449951,6474.919922,6447.200195,18168.269531


In [5]:
# Find out number of missing values for each Stock Exchanges
df.isna().sum()

bse       310
spx       195
dax       148
ftse      173
nikkei    346
dtype: int64

In [6]:
df = df.iloc[1:, :]
df.head()

,bse,spx,dax,ftse,nikkei
Date,,,,,
2000-01-03,5375.109863,1455.219971,6750.759766,NaN,NaN
2000-01-04,5491.009766,1399.420044,6586.950195,6665.899902,19002.859375
2000-01-05,5357.000000,1402.109985,6502.069824,6535.899902,18542.550781
2000-01-06,5421.529785,1403.449951,6474.919922,6447.200195,18168.269531
2000-01-07,5414.479980,1441.469971,6780.959961,6504.799805,18193.410156


In [7]:
# Setting the frequency of the data
# h = hourly, w = Weekly, d = daily, m = monthly, b = business days
df=df.asfreq('b')

In [8]:
df=df.fillna(method='ffill') # Front-Filling any missing value
df = df.fillna(method='bfill') # Back-Filling any missing value
# df = df.fillna(value=df.mean()) # Fill with mean value
df

,bse,spx,dax,ftse,nikkei
Date,,,,,
2000-01-03,5375.109863,1455.219971,6750.759766,6665.899902,19002.859375
2000-01-04,5491.009766,1399.420044,6586.950195,6665.899902,19002.859375
2000-01-05,5357.000000,1402.109985,6502.069824,6535.899902,18542.550781
2000-01-06,5421.529785,1403.449951,6474.919922,6447.200195,18168.269531
2000-01-07,5414.479980,1441.469971,6780.959961,6504.799805,18193.410156
...,...,...,...,...,...
2022-05-25,53749.261719,3978.729980,14007.929688,7522.799805,26677.800781
2022-05-26,54252.531250,4057.840088,14231.290039,7564.899902,26604.839844
2022-05-27,54884.660156,4158.240234,14462.190430,7585.500000,26781.679688


In [9]:
# Displaying the first and last 5 elements to make sure the data was scrapped correctly
print(df.head())
print(df.tail())

                    bse          spx          dax         ftse        nikkei
Date                                                                        
2000-01-03  5375.109863  1455.219971  6750.759766  6665.899902  19002.859375
2000-01-04  5491.009766  1399.420044  6586.950195  6665.899902  19002.859375
2000-01-05  5357.000000  1402.109985  6502.069824  6535.899902  18542.550781
2000-01-06  5421.529785  1403.449951  6474.919922  6447.200195  18168.269531
2000-01-07  5414.479980  1441.469971  6780.959961  6504.799805  18193.410156
                     bse          spx           dax         ftse        nikkei
Date                                                                          
2022-05-25  53749.261719  3978.729980  14007.929688  7522.799805  26677.800781
2022-05-26  54252.531250  4057.840088  14231.290039  7564.899902  26604.839844
2022-05-27  54884.660156  4158.240234  14462.190430  7585.500000  26781.679688
2022-05-30  55925.738281  4158.240234  14575.980469  7600.100098  

4 - Export Database

In [10]:
df.to_csv("stock-indexes.csv")